In [1]:
#@title Fruit Dataset Loader
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt

ds = load_dataset("Suru/Fruit")

print(ds)


c:\Project\MESINLEARNING\fruit-image-classification-cnn\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 16/16 [00:00<00:00, 1111.35 examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 118
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 34
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 16
    })
})


In [2]:
#@title Data Preprocessing
import tensorflow as tf

IMG_SIZE = 224

def preprocess(example):
    img = example["image"].resize((IMG_SIZE, IMG_SIZE))
    img = np.array(img) / 255.0
    return img, example["label"]

X_train, y_train = zip(*[preprocess(x) for x in ds["train"]])
X_val, y_val     = zip(*[preprocess(x) for x in ds["validation"]])
X_test, y_test   = zip(*[preprocess(x) for x in ds["test"]])

X_train = np.array(X_train)
X_val   = np.array(X_val)
X_test  = np.array(X_test)


c:\Project\MESINLEARNING\fruit-image-classification-cnn\venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [5]:
#@title Feature Extraction using MobileNetV2
from tensorflow.keras.applications import MobileNetV2

base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3),
    pooling="avg"
)

base_model.trainable = False

X_train_feat = base_model.predict(X_train)
X_val_feat   = base_model.predict(X_val)
X_test_feat  = base_model.predict(X_test)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 400ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step


In [6]:
#@title Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_feat, y_train)

y_pred = clf.predict(X_test_feat)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00         9

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16



In [7]:
#@title Save the Model
import joblib

joblib.dump(clf, "../model/classifier.pkl")
base_model.save("../model/mobilenet_feature_extractor.h5")
